In [3]:
import numpy as np
import pandas as pd
from scipy import io
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE

# Load data
y_tr = pd.read_csv('tox21_labels_train.csv.gz', index_col=0, compression="gzip")
y_te = pd.read_csv('tox21_labels_test.csv.gz', index_col=0, compression="gzip")
x_tr_dense = pd.read_csv('tox21_dense_train.csv.gz', index_col=0, compression="gzip").values
x_te_dense = pd.read_csv('tox21_dense_test.csv.gz', index_col=0, compression="gzip").values
x_tr_sparse = io.mmread('tox21_sparse_train.mtx.gz').tocsc()
x_te_sparse = io.mmread('tox21_sparse_test.mtx.gz').tocsc()

# Filter and concatenate dense + sparse features
sparse_col_idx = ((x_tr_sparse > 0).mean(0) > 0.05).A.ravel()
x_tr = np.hstack([x_tr_dense, x_tr_sparse[:, sparse_col_idx].A])
x_te = np.hstack([x_te_dense, x_te_sparse[:, sparse_col_idx].A])

# Standardize features with tanh scaling
scaler = StandardScaler()
x_tr = np.tanh(scaler.fit_transform(x_tr))
x_te = np.tanh(scaler.transform(x_te))

# Learning rate decay function
def lr_schedule(epoch, lr):
    return lr * 0.95 if epoch > 10 else lr

# Create a shared multi-task model architecture with LeakyReLU
def create_model(input_dim):
    model = Sequential([
        Dense(256, input_dim=input_dim, kernel_regularizer=l2(1e-5)),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.4),

        Dense(128, kernel_regularizer=l2(1e-5)),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),

        Dense(1, activation='sigmoid')  # Single output for binary classification
    ])
    return model

# Early stopping and learning rate scheduler
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train and evaluate the model for each task independently
auc_scores = []

for i, target in enumerate(y_tr.columns):
    print(f"\nTraining on assay: {target}")

    # Select valid rows for the current task
    valid_rows = np.isfinite(y_tr[target]).values
    x_target, y_target = x_tr[valid_rows], y_tr[target][valid_rows]

    # Apply SMOTE to handle class imbalance
    smote = SMOTE(random_state=42)
    x_balanced, y_balanced = smote.fit_resample(x_target, y_target)

    # Split into train/validation sets
    x_train, x_val, y_train, y_val = train_test_split(
        x_balanced, y_balanced, test_size=0.2, random_state=42
    )

    # Create and compile a new model and optimizer for the current task
    model = create_model(input_dim=x_tr.shape[1])
    optimizer = Adam(learning_rate=1e-3)  # Create a new optimizer instance
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])

    # Train the model
    model.fit(
        x_train, y_train, validation_data=(x_val, y_val),
        epochs=100, batch_size=32, callbacks=[early_stopping, reduce_lr], verbose=2
    )

    # Evaluate on the test set
    valid_test_rows = np.isfinite(y_te[target]).values
    y_test = y_te[target][valid_test_rows].values
    p_test = model.predict(x_te[valid_test_rows]).ravel()

    # Calculate the AUC score for the current task
    auc = roc_auc_score(y_test, p_test)
    auc_scores.append(auc)
    print(f"{target}: Test AUC = {auc:.3f}")

# Calculate and print the average AUC score
avg_auc = np.mean(auc_scores)
print(f"\nAverage Test AUC: {avg_auc:.3f}")



Training on assay: NR.AhR


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
373/373 - 8s - 21ms/step - AUC: 0.9036 - loss: 0.4028 - val_AUC: 0.9506 - val_loss: 0.2895 - learning_rate: 0.0010
Epoch 2/100
373/373 - 5s - 13ms/step - AUC: 0.9375 - loss: 0.3252 - val_AUC: 0.9591 - val_loss: 0.2625 - learning_rate: 0.0010
Epoch 3/100
373/373 - 1s - 3ms/step - AUC: 0.9497 - loss: 0.2928 - val_AUC: 0.9670 - val_loss: 0.2374 - learning_rate: 0.0010
Epoch 4/100
373/373 - 1s - 3ms/step - AUC: 0.9559 - loss: 0.2739 - val_AUC: 0.9728 - val_loss: 0.2162 - learning_rate: 0.0010
Epoch 5/100
373/373 - 1s - 2ms/step - AUC: 0.9656 - loss: 0.2444 - val_AUC: 0.9696 - val_loss: 0.2261 - learning_rate: 0.0010
Epoch 6/100
373/373 - 1s - 3ms/step - AUC: 0.9680 - loss: 0.2356 - val_AUC: 0.9780 - val_loss: 0.1925 - learning_rate: 0.0010
Epoch 7/100
373/373 - 1s - 2ms/step - AUC: 0.9719 - loss: 0.2215 - val_AUC: 0.9760 - val_loss: 0.2034 - learning_rate: 0.0010
Epoch 8/100
373/373 - 1s - 3ms/step - AUC: 0.9742 - loss: 0.2124 - val_AUC: 0.9788 - val_loss: 0.1917 - learning_rat

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
464/464 - 9s - 20ms/step - AUC: 0.9099 - loss: 0.3787 - val_AUC: 0.9637 - val_loss: 0.2642 - learning_rate: 0.0010
Epoch 2/100
464/464 - 3s - 7ms/step - AUC: 0.9576 - loss: 0.2711 - val_AUC: 0.9776 - val_loss: 0.2087 - learning_rate: 0.0010
Epoch 3/100
464/464 - 1s - 2ms/step - AUC: 0.9737 - loss: 0.2175 - val_AUC: 0.9849 - val_loss: 0.1709 - learning_rate: 0.0010
Epoch 4/100
464/464 - 1s - 3ms/step - AUC: 0.9815 - loss: 0.1834 - val_AUC: 0.9865 - val_loss: 0.1582 - learning_rate: 0.0010
Epoch 5/100
464/464 - 1s - 2ms/step - AUC: 0.9838 - loss: 0.1723 - val_AUC: 0.9928 - val_loss: 0.1166 - learning_rate: 0.0010
Epoch 6/100
464/464 - 1s - 3ms/step - AUC: 0.9870 - loss: 0.1549 - val_AUC: 0.9908 - val_loss: 0.1254 - learning_rate: 0.0010
Epoch 7/100
464/464 - 1s - 2ms/step - AUC: 0.9897 - loss: 0.1386 - val_AUC: 0.9934 - val_loss: 0.1148 - learning_rate: 0.0010
Epoch 8/100
464/464 - 1s - 2ms/step - AUC: 0.9909 - loss: 0.1318 - val_AUC: 0.9955 - val_loss: 0.1000 - learning_rate

...

Epoch 1/100
399/399 - 9s - 22ms/step - AUC: 0.8603 - loss: 0.4773 - val_AUC: 0.9435 - val_loss: 0.3123 - learning_rate: 0.0010
Epoch 2/100
399/399 - 4s - 10ms/step - AUC: 0.9307 - loss: 0.3419 - val_AUC: 0.9706 - val_loss: 0.2402 - learning_rate: 0.0010
Epoch 3/100
399/399 - 1s - 2ms/step - AUC: 0.9538 - loss: 0.2812 - val_AUC: 0.9780 - val_loss: 0.1960 - learning_rate: 0.0010
Epoch 4/100
399/399 - 1s - 3ms/step - AUC: 0.9640 - loss: 0.2464 - val_AUC: 0.9762 - val_loss: 0.2123 - learning_rate: 0.0010
Epoch 5/100
399/399 - 1s - 3ms/step - AUC: 0.9741 - loss: 0.2126 - val_AUC: 0.9859 - val_loss: 0.1622 - learning_rate: 0.0010
Epoch 6/100
399/399 - 1s - 3ms/step - AUC: 0.9769 - loss: 0.2006 - val_AUC: 0.9899 - val_loss: 0.1309 - learning_rate: 0.0010
Epoch 7/100
399/399 - 1s - 3ms/step - AUC: 0.9821 - loss: 0.1777 - val_AUC: 0.9890 - val_loss: 0.1461 - learning_rate: 0.0010
Epoch 8/100
399/399 - 1s - 3ms/step - AUC: 0.9821 - loss: 0.1770 - val_AUC: 0.9896 - val_loss: 0.1316 - learning_rat

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
319/319 - 9s - 27ms/step - AUC: 0.8912 - loss: 0.4270 - val_AUC: 0.9404 - val_loss: 0.3193 - learning_rate: 0.0010
Epoch 2/100
319/319 - 4s - 13ms/step - AUC: 0.9311 - loss: 0.3423 - val_AUC: 0.9543 - val_loss: 0.2897 - learning_rate: 0.0010
Epoch 3/100
319/319 - 1s - 2ms/step - AUC: 0.9458 - loss: 0.3047 - val_AUC: 0.9651 - val_loss: 0.2481 - learning_rate: 0.0010
Epoch 4/100
319/319 - 1s - 4ms/step - AUC: 0.9516 - loss: 0.2880 - val_AUC: 0.9689 - val_loss: 0.2290 - learning_rate: 0.0010
Epoch 5/100
319/319 - 1s - 2ms/step - AUC: 0.9601 - loss: 0.2623 - val_AUC: 0.9717 - val_loss: 0.2170 - learning_rate: 0.0010
Epoch 6/100
319/319 - 1s - 4ms/step - AUC: 0.9654 - loss: 0.2451 - val_AUC: 0.9627 - val_loss: 0.2546 - learning_rate: 0.0010
Epoch 7/100
319/319 - 1s - 4ms/step - AUC: 0.9690 - loss: 0.2344 - val_AUC: 0.9796 - val_loss: 0.1884 - learning_rate: 0.0010
Epoch 8/100
319/319 - 1s - 3ms/step - AUC: 0.9730 - loss: 0.2182 - val_AUC: 0.9782 - val_loss: 0.1924 - learning_rat

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
417/417 - 7s - 18ms/step - AUC: 0.8783 - loss: 0.4435 - val_AUC: 0.9542 - val_loss: 0.2797 - learning_rate: 0.0010
Epoch 2/100
417/417 - 1s - 2ms/step - AUC: 0.9374 - loss: 0.3214 - val_AUC: 0.9583 - val_loss: 0.2620 - learning_rate: 0.0010
Epoch 3/100
417/417 - 1s - 3ms/step - AUC: 0.9567 - loss: 0.2694 - val_AUC: 0.9767 - val_loss: 0.2074 - learning_rate: 0.0010
Epoch 4/100
417/417 - 1s - 2ms/step - AUC: 0.9680 - loss: 0.2329 - val_AUC: 0.9827 - val_loss: 0.1701 - learning_rate: 0.0010
Epoch 5/100
417/417 - 1s - 3ms/step - AUC: 0.9735 - loss: 0.2126 - val_AUC: 0.9860 - val_loss: 0.1563 - learning_rate: 0.0010
Epoch 6/100
417/417 - 1s - 3ms/step - AUC: 0.9794 - loss: 0.1881 - val_AUC: 0.9884 - val_loss: 0.1358 - learning_rate: 0.0010
Epoch 7/100
417/417 - 1s - 3ms/step - AUC: 0.9820 - loss: 0.1781 - val_AUC: 0.9892 - val_loss: 0.1421 - learning_rate: 0.0010
Epoch 8/100
417/417 - 1s - 2ms/step - AUC: 0.9859 - loss: 0.1564 - val_AUC: 0.9921 - val_loss: 0.1125 - learning_rate